### ErrorLogAnalysis 

In [0]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from datetime import datetime

spark = SparkSession.builder.appName("ErrorLogAnalysis").getOrCreate()
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("ErrorLogAnalysis")
logger = logging.getLogger(f"{timestamp} [INFO] ErrorLogAnalysis - Inicializando...")

logger.info(f"{timestamp} [INFO] Carregando tabelas: 'equpment_failure_sensors_loaded'; 'df_sensors_loaded'; 'equipment_loaded'") 
df_logs = spark.read.parquet("dbfs:/FileStore/workflow/equpment_failure_sensors_loaded.parquet")
df_sensors = spark.read.parquet("dbfs:/FileStore/workflow/df_sensors_loaded.parquet")
df_equipment = spark.read.parquet("dbfs:/FileStore/workflow/equipment_loaded.parquet")

df_logs_alias = df_logs.alias("logs")
df_sensors_alias = df_sensors.alias("sensors")
df_equipment_alias = df_equipment.alias("equipment")
logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")

2024-04-26 19:36:32,245 - INFO - 2024-04-26_19-36-32 [INFO] Carregando tabelas: 'equpment_failure_sensors_loaded'; 'df_sensors_loaded'; 'equipment_loaded'
2024-04-26 19:36:34,799 - INFO - 2024-04-26_19-36-32 [INFO] Ação realizada com sucesso!


In [0]:
logger.info(f"{timestamp} [INFO] Inciando Transformação de dados: ErrorLogAnalysis ")
try: 
# Join
    failures_with_equipment = df_logs_alias \
        .join(df_sensors_alias, df_logs_alias["sensor_id"] == df_sensors_alias["sensor_id"]) \
        .join(df_equipment_alias, df_sensors_alias["equipment_id"] == df_equipment_alias["equipment_id"])
except Exception as e:
    logger.error(f"{timestamp} [ERROR] Erro na Transformação dos dados: {e}.")
    
# Filtrar apenas linhas com 'status' igual a 'ERROR'
failures_with_equipment = failures_with_equipment.filter(col("status") == "ERROR")

# 'count' na coluna 'name' from dataframe 'equipment'
equipment_failure_counts = failures_with_equipment \
    .groupBy(df_equipment_alias["name"]) \
    .count() \
    .orderBy(col("count").desc())

logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!")

2024-04-26 19:36:37,456 - INFO - 2024-04-26_19-36-32 [INFO] Inciando Transformação de dados: ErrorLogAnalysis 
2024-04-26 19:36:37,589 - INFO - 2024-04-26_19-36-32 [INFO] Ação realizada com sucesso!


In [0]:
# Criando dataframe equipment_failure_counts
equipment_failure_counts = equipment_failure_counts \
    .withColumnRenamed("name", "equipment_name") \
    .withColumnRenamed("count", "equipment_failure_count")
most_failures_equipment = equipment_failure_counts.first()

logger.info(f"{timestamp} [WARNING] Equipamento com maior numero de falhas: {most_failures_equipment} ") 
logger.info(f"{timestamp} [INFO] Salvando tabela: {equipment_failure_counts}...") 
# Salvando a tabela no databricks.
equipment_failure_counts.write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("most_failures_equipment")

logger.info(f"{timestamp} [INFO] Ação realizada com sucesso!") 
logger.info(f"{timestamp} [INFO] ErrorLogAnalysis - Finalizado com sucesso!") 
display(equipment_failure_counts)


2024-04-26 19:36:49,145 - INFO - 2024-04-26_19-36-32 [WARNING] Equipamento com maior numero de falhas: DataFrame[equipment_name: string, equipment_failure_count: bigint] 
2024-04-26 19:36:49,160 - INFO - 2024-04-26_19-36-32 [INFO] Salvando tabelas: DataFrame[equipment_name: string, equipment_failure_count: bigint]...
2024-04-26 19:37:25,874 - INFO - 2024-04-26_19-36-32 [INFO] Ação realizada com sucesso!
2024-04-26 19:37:25,876 - INFO - 2024-04-26_19-36-32 [INFO] ErrorLogAnalysis - Finalizado com sucesso!


equipment_name equipment_failure_count 2C195700 343800 9AD15F7E 343526 98B84035 342870 78FFAD0C 341033 4E834E81 340549 43B81579 340188 E1AD07D4 340169 ADE40E7F 339698 3329175B 339561 86083278 338766 5310B9D7 337033 CF304D24 334843 09C37FB8 334615 E54B5C3A 332824